In [3]:
import pandas as pd
import numpy as np

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, learning_curve
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [41]:
# Loading the data
train_df = pd.read_csv("/kaggle/input/joined-isic-optimized-87-df/joined_train_df_optimized.csv")
test_df = pd.read_csv("/kaggle/input/joined-test-87-isic/joined_test_df_optimized.csv")

In [42]:
print(train_df.shape)
print(test_df.shape)

(29488, 1563)
(2293, 1563)


In [43]:
train_df["disease_label"].unique()

array(['Melanocytic_nevus', 'Basal_cell_carcinoma',
       'Squamous_cell_carcinoma', 'Vascular_lesion', 'Dermatofibroma',
       'Actinic_keratosis', 'Benign_keratosis', 'Melanoma'], dtype=object)

In [44]:
test_df["disease_label"].unique()

array(['Melanocytic_nevus', 'Benign_keratosis', 'Basal_cell_carcinoma',
       'Melanoma', 'Squamous_cell_carcinoma', 'Actinic_keratosis',
       'Vascular_lesion', 'Dermatofibroma'], dtype=object)

In [45]:
test_df.isna().sum()

Melanoma                0
Melanocytic_nevus       0
Basal_cell_carcinoma    0
Actinic_keratosis       0
Benign_keratosis        0
                       ..
feature_1531            0
feature_1532            0
feature_1533            0
feature_1534            0
feature_1535            0
Length: 1563, dtype: int64

In [46]:
train_df.isna().sum()

Melanoma                0
Melanocytic_nevus       0
Basal_cell_carcinoma    0
Actinic_keratosis       0
Benign_keratosis        0
                       ..
feature_1531            0
feature_1532            0
feature_1533            0
feature_1534            0
feature_1535            0
Length: 1563, dtype: int64

In [47]:
test_df.head()

,Melanoma,Melanocytic_nevus,Basal_cell_carcinoma,Actinic_keratosis,Benign_keratosis,Dermatofibroma,Vascular_lesion,Squamous_cell_carcinoma,image_path,site_anterior torso,...,feature_1526,feature_1527,feature_1528,feature_1529,feature_1530,feature_1531,feature_1532,feature_1533,feature_1534,feature_1535
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,/kaggle/input/isic19/ISIC 2019/ISIC_2019_Train...,1.0,...,-0.358680,-0.275448,-0.219601,-0.079318,0.243864,0.161635,-0.135520,-0.225854,-0.295673,0.256497
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,/kaggle/input/isic19/ISIC 2019/ISIC_2019_Train...,0.0,...,0.224395,0.358083,-0.093568,0.005221,-0.162681,0.383933,0.509697,0.352567,0.180713,0.488324
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,/kaggle/input/isic19/ISIC 2019/ISIC_2019_Train...,1.0,...,-0.131074,0.343094,-0.257487,0.175053,-0.265037,0.326875,-0.068459,-0.093237,0.240568,-0.235423
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,/kaggle/input/isic19/ISIC 2019/ISIC_2019_Train...,1.0,...,-0.422874,-0.063306,-0.251134,-0.531468,0.200831,0.284164,-0.155972,-0.101167,-0.100669,0.093928
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,/kaggle/input/isic19/ISIC 2019/ISIC_2019_Train...,0.0,...,-0.335398,0.242917,-0.115023,0.328015,-0.226987,0.445545,0.189325,-0.297067,0.428755,-0.079604


In [48]:
train_df.head()

,Melanoma,Melanocytic_nevus,Basal_cell_carcinoma,Actinic_keratosis,Benign_keratosis,Dermatofibroma,Vascular_lesion,Squamous_cell_carcinoma,image_path,site_anterior torso,...,feature_1526,feature_1527,feature_1528,feature_1529,feature_1530,feature_1531,feature_1532,feature_1533,feature_1534,feature_1535
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,/kaggle/input/isic19/ISIC 2019/ISIC_2019_Train...,0.0,...,-0.381115,0.042536,-0.221526,-0.465344,0.216053,0.265413,-0.112709,-0.198899,-0.056672,0.053271
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,/kaggle/input/isic19/ISIC 2019/ISIC_2019_Train...,0.0,...,0.051138,0.402260,-0.500111,0.215994,-0.128300,0.237245,0.067042,-0.008089,0.299901,-0.179578
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,/kaggle/input/isic19/ISIC 2019/ISIC_2019_Train...,1.0,...,-0.347500,-0.060503,-0.278919,-0.451418,0.150784,0.244543,-0.189390,-0.029913,-0.183265,0.154720
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,/kaggle/input/augmented-final-2/Squamous_cell_...,0.0,...,0.093930,-0.117644,0.286670,-0.218164,0.048146,0.212154,-0.132744,0.460772,-0.104400,0.129038
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,/kaggle/input/augmented-final-2/Vascular_lesio...,0.0,...,0.094021,-0.235975,0.142994,0.097290,-0.164727,-0.069371,-0.329767,0.255132,-0.191676,-0.610850


In [49]:
print("Train_shape: ", train_df.shape)
print("Test_shape: ", test_df.shape)

Train_shape:  (29488, 1563)
Test_shape:  (2293, 1563)


In [50]:
# Droping the one-hot encoded label columns (Because the "Disease Label" column is enough to train the RF model)
label_cols = [
    'Melanoma', 'Melanocytic_nevus', 'Basal_cell_carcinoma', 
    'Actinic_keratosis', 'Benign_keratosis', 'Dermatofibroma', 
    'Vascular_lesion', 'Squamous_cell_carcinoma'
]

train_df = train_df.drop(columns=label_cols)
test_df = test_df.drop(columns=label_cols)

In [17]:
print("Train_shape: ", train_df.shape)
print("Test_shape: ", test_df.shape)

Train_shape:  (29488, 1555)
Test_shape:  (2293, 1555)


In [18]:
train_df.head()

,image_path,site_anterior torso,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_posterior torso,site_upper extremity,sex_female,sex_male,...,feature_1526,feature_1527,feature_1528,feature_1529,feature_1530,feature_1531,feature_1532,feature_1533,feature_1534,feature_1535
0,/kaggle/input/isic19/ISIC 2019/ISIC_2019_Train...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.381115,0.042536,-0.221526,-0.465344,0.216053,0.265413,-0.112709,-0.198899,-0.056672,0.053271
1,/kaggle/input/isic19/ISIC 2019/ISIC_2019_Train...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.051138,0.402260,-0.500111,0.215994,-0.128300,0.237245,0.067042,-0.008089,0.299901,-0.179578
2,/kaggle/input/isic19/ISIC 2019/ISIC_2019_Train...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-0.347500,-0.060503,-0.278919,-0.451418,0.150784,0.244543,-0.189390,-0.029913,-0.183265,0.154720
3,/kaggle/input/augmented-final-2/Squamous_cell_...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.093930,-0.117644,0.286670,-0.218164,0.048146,0.212154,-0.132744,0.460772,-0.104400,0.129038
4,/kaggle/input/augmented-final-2/Vascular_lesio...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.094021,-0.235975,0.142994,0.097290,-0.164727,-0.069371,-0.329767,0.255132,-0.191676,-0.610850


In [19]:
# Seting the X and y
X_train = train_df.drop(columns=['disease_label', 'image_path'])
y_train = train_df['disease_label']

X_test = test_df.drop(columns=['disease_label', 'image_path'])
y_test = test_df['disease_label']

In [20]:
X_train.head()

,site_anterior torso,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_posterior torso,site_upper extremity,sex_female,sex_male,age_1-10,...,feature_1526,feature_1527,feature_1528,feature_1529,feature_1530,feature_1531,feature_1532,feature_1533,feature_1534,feature_1535
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,...,-0.381115,0.042536,-0.221526,-0.465344,0.216053,0.265413,-0.112709,-0.198899,-0.056672,0.053271
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,...,0.051138,0.402260,-0.500111,0.215994,-0.128300,0.237245,0.067042,-0.008089,0.299901,-0.179578
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,...,-0.347500,-0.060503,-0.278919,-0.451418,0.150784,0.244543,-0.189390,-0.029913,-0.183265,0.154720
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0,...,0.093930,-0.117644,0.286670,-0.218164,0.048146,0.212154,-0.132744,0.460772,-0.104400,0.129038
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0,...,0.094021,-0.235975,0.142994,0.097290,-0.164727,-0.069371,-0.329767,0.255132,-0.191676,-0.610850


In [22]:
print("Data shapes:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

Data shapes:
X_train: (29488, 1553), y_train: (29488,)
X_test: (2293, 1553), y_test: (2293,)


In [23]:
rf_baseline = RandomForestClassifier(random_state=42)

In [24]:
rf_baseline.fit(X_train, y_train)
print("Training accuracy: {:.3f}".format(rf_baseline.score(X_train, y_train)))
print("Test accuracy: {:.3f}".format(rf_baseline.score(X_test, y_test)))

Training accuracy: 0.993
Test accuracy: 0.870


In [25]:
import joblib

# Saving the model
joblib.dump(rf_baseline, 'rf_baseline_model.pkl')

['rf_baseline_model.pkl']

# Random Search

In [27]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import optuna
import time
import warnings
warnings.filterwarnings('ignore')

In [28]:
# Defining the objective function for Optuna
def objective(trial):
    # Defining the hyperparameter search space
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300, step=25),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False]),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample', None]),
        'random_state': 42,
        'n_jobs': -1
    }
    
    # Creating and evaluating the model with a 3-fold CV for speed
    rf = RandomForestClassifier(**params)
    
    # Using a 3-fold CV for a faster evaluation
    cv_scores = cross_val_score(rf, X_train, y_train, cv=3, scoring='accuracy', n_jobs=-1)
    
    return cv_scores.mean()

In [29]:
# Creating a study with aggressive pruning for speed
print("\nStarting Optuna hyperparameter optimization...")
print("=" * 60)

# Using the TPE sampler for a faster convergence
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(
    direction='maximize',
    sampler=sampler,
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=1)
)

[I 2025-06-05 21:39:08,568] A new study created in memory with name: no-name-7daa0225-c9a5-42fc-8551-68aa63cdef6c



Starting Optuna hyperparameter optimization...


In [30]:
# Recording the start time
start_time = time.time()

In [31]:
# Optimizing (reducing the trials for speed)
study.optimize(objective, n_trials=50, timeout=300)  # 5-minute timeout

[I 2025-06-05 21:40:32,305] Trial 0 finished with value: 0.9858925951850569 and parameters: {'n_estimators': 150, 'max_depth': 20, 'min_samples_split': 15, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'bootstrap': True, 'class_weight': None}. Best is trial 0 with value: 0.9858925951850569.
[I 2025-06-05 21:41:26,025] Trial 1 finished with value: 0.9892498930766634 and parameters: {'n_estimators': 275, 'max_depth': 6, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': False, 'class_weight': None}. Best is trial 1 with value: 0.9892498930766634.
[I 2025-06-05 22:42:51,873] Trial 2 finished with value: 0.9853839033364049 and parameters: {'n_estimators': 175, 'max_depth': 17, 'min_samples_split': 5, 'min_samples_leaf': 6, 'max_features': None, 'bootstrap': True, 'class_weight': 'balanced_subsample'}. Best is trial 1 with value: 0.9892498930766634.


In [32]:
# Recording the end time
end_time = time.time()
optimization_time = end_time - start_time

In [33]:
print(f"\nOptimization completed in {optimization_time:.2f} seconds ({optimization_time/60:.2f} minutes)")
print(f"Number of completed trials: {len(study.trials)}")
print(f"Best cross-validation accuracy: {study.best_value:.4f}")


Optimization completed in 3821.48 seconds (63.69 minutes)
Number of completed trials: 3
Best cross-validation accuracy: 0.9892


In [34]:
# Geting the best parameters
best_params = study.best_params
print(f"\nBest hyperparameters:")
print("=" * 40)
for param, value in best_params.items():
    print(f"{param}: {value}")


Best hyperparameters:
n_estimators: 275
max_depth: 6
min_samples_split: 5
min_samples_leaf: 2
max_features: log2
bootstrap: False
class_weight: None


In [35]:
# Training the final model with the best parameters
print("\nTraining final model with best parameters...")
best_rf = RandomForestClassifier(**best_params, random_state=42, n_jobs=-1)
best_rf.fit(X_train, y_train)


Training final model with best parameters...


RandomForestClassifier(bootstrap=False, max_depth=6, max_features='log2',
                       min_samples_leaf=2, min_samples_split=5,
                       n_estimators=275, n_jobs=-1, random_state=42)

In [36]:
# Evaluating the final model
final_train_acc = best_rf.score(X_train, y_train)
final_test_acc = best_rf.score(X_test, y_test)

In [37]:
print(f"\nFinal Results:")
print("=" * 40)
print(f"Training accuracy: {final_train_acc:.4f}")
print(f"Test accuracy: {final_test_acc:.4f}")
print(f"Improvement over baseline: {final_test_acc - 0.866:.4f}")


Final Results:
Training accuracy: 0.9908
Test accuracy: 0.8727
Improvement over baseline: 0.0067


In [38]:
# Making predictions
y_pred = best_rf.predict(X_test)

In [39]:
# Saving the best Model
joblib.dump(best_rf, 'best_rf_model.pkl')

# Classification report
print(f"\nClassification Report:")
print("=" * 60)
print(classification_report(y_test, y_pred))


Classification Report:
                         precision    recall  f1-score   support

      Actinic_keratosis       0.72      0.68      0.70       125
   Basal_cell_carcinoma       0.91      0.92      0.91       301
       Benign_keratosis       0.88      0.77      0.82       241
         Dermatofibroma       0.94      0.85      0.89        34
      Melanocytic_nevus       0.88      0.96      0.92      1071
               Melanoma       0.91      0.76      0.83       422
Squamous_cell_carcinoma       0.59      0.62      0.60        71
        Vascular_lesion       0.96      0.96      0.96        28

               accuracy                           0.87      2293
              macro avg       0.85      0.82      0.83      2293
           weighted avg       0.87      0.87      0.87      2293

